## Estimating individual systematic contributions

The effect of any individual systematic uncertainty is somewhat complicated by it's mutual covariance with the POI (i.e., the W branching fractions) and any other systematic uncertainty.  To get a rough idea of the percent-wise contribution to the total uncertainty from each individual systematic, I use the following scheme:

   * the fit is carried out as in the nominal case and $\sigma_{0}$ is estimated
   * the fit is carried out for for each of the $n$ nuisance parameters $\sigma_{theta}$
   * the difference between the nominal case and the $n-1$ case is calculated,
   * this quantity is normalized to $\sum_{\theta} \sigma_{\theta}^{2}$

In [1]:
## imports and configuration
%cd '/home/naodell/work/wbr/analysis'
#%load_ext autoreload

from multiprocessing import Pool
from functools import partial
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy.optimize import minimize
from tqdm import tqdm_notebook, trange

import scripts.plot_tools as pt
import scripts.fit_helpers as fh

np.set_printoptions(precision=3)
rc_params = {
             'figure.figsize': (10, 10),
             'axes.labelsize': 20,
             'axes.facecolor': 'white',
             'axes.titlesize':'x-large',
             'legend.fontsize': 20,
             'xtick.labelsize':18,
             'ytick.labelsize':18,
             'font.size':18,
             'font.sans-serif':['Arial', 'sans-serif'],
             'mathtext.sf':'Arial',
             'lines.markersize':8.,
             'lines.linewidth':2.5,
            }
matplotlib.rcParams.update(rc_params)

%connect_info

/home/naodell/work/wbr/analysis
{
  "shell_port": 57879,
  "iopub_port": 60185,
  "stdin_port": 51935,
  "control_port": 43631,
  "hb_port": 51459,
  "ip": "127.0.0.1",
  "key": "2fb65ad7-8ca0f5995cba786b925b5141",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": ""
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-f44b9ef1-8fa9-4eaa-9cef-7163d7efbc74.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
# configure, get the input data, and do any additional processing that is needed
processes = ['ttbar', 't', 'ww', 'wjets', 'zjets_alt', 'diboson', 'fakes'] 
selections = [
              'ee',  'mumu',  
              'emu', 
              'mutau', 'etau', 
              'mu4j', 'e4j'
             ]
plot_labels = fh.fancy_labels

# initialize fit data
infile = open('local_data/fit_data_postfit.pkl', 'rb')
fit_data = pickle.load(infile)
infile.close()

# get fit parameters
parameters = pd.read_csv('data/model_parameters_asimov.csv', index_col='name')

In [3]:
# prepare Asimov dataset
asimov_data = {cat:fit_data.mixture_model(parameters.val_init.values, cat) for cat in fit_data._model_data.keys()}

In [4]:
# initialize veto list
fit_data.veto_list = [
    # baseline
    #'ee_cat_gt2_eq1_b', 'ee_cat_gt2_gt2_b', 
    #'mumu_cat_gt2_eq1_b', 'mumu_cat_gt2_gt2_b', 
    #'emu_cat_gt2_eq1_a', 'emu_cat_gt2_gt2_a', 
    #'etau_cat_eq2_eq1', 'etau_cat_gt3_eq1', 'etau_cat_eq2_gt2', 'etau_cat_gt3_gt2', 
    #'mutau_cat_eq2_eq1', 'mutau_cat_gt3_eq1', 'mutau_cat_eq2_gt2', 'mutau_cat_gt3_gt2', 
    #'e4j_cat_gt4_eq1', 'e4j_cat_gt4_gt2',
    #'mu4j_cat_gt4_eq1', 'mu4j_cat_gt4_gt2', 
    'e4j_cat_eq3_gt2', 'mu4j_cat_eq3_gt2',
    
    # e/mu DY CR
    'ee_cat_gt2_eq0',  'mumu_cat_gt2_eq0', 
    
    # e+mu additional ttbar
    #'emu_cat_gt2_eq0', 'emu_cat_eq1_eq0_a', 'emu_cat_eq1_eq1_a', 
    
    # e+mu WW
    #'emu_cat_eq0_eq0_a', 
    
    # e/mu+tau additional CR
    #'mutau_cat_eq0_eq0', 'mutau_cat_eq1_eq0', 
    #'mutau_cat_gt2_eq0', 'mutau_cat_eq1_eq1', 
    #'etau_cat_eq0_eq0', 'etau_cat_eq1_eq0', 
    #'etau_cat_gt2_eq0', 'etau_cat_eq1_eq1', 
]

In [5]:
# fit configuration #

# minimizer options
#steps = 4*[1e-4, ] 
#steps += list(0.05*fit_data._perr_init[4: fit_data._nnorm + 7])
#steps += list(0.05*fit_data._perr_init[fit_data._npoi + fit_data._nnorm:])
#steps = np.array(steps)
min_options = dict(
                   #finite_diff_rel_step=steps,
                   #verbose=3,
                   #eps=1e-9, 
                   gtol = 1e-3,
                   disp = True
                  )

# configure the objective
sample = asimov_data
fobj = partial(fit_data.objective,
               data = sample,
               do_bb_lite = True,
               lu_test = None
              )

fobj_jac = partial(fit_data.objective_jacobian,
                   data = sample,
                   do_bb_lite = True,
                   lu_test = None
                  )


In [6]:
# n-1 systematics

# initialize parameter data
fit_data._pmask = parameters['active'].values.astype(bool)
fit_data._pval_init = parameters['val_init'].values.copy()

errs = []
impacts = []
iparam = 4
mask = fit_data._pmask
#mask[17:] = False
p_init = fit_data._pval_fit
p_fit = parameters.val_fit.values
for pname, pdata in tqdm_notebook(parameters.iloc[4:].iterrows(), total=parameters.active.sum() - 4):
    if not pdata.active:
        iparam += 1
        continue
        
    print(iparam, pname, pdata.val_init, pdata.err_init, pdata.val_fit, pdata.err_fit, fobj(p_init[mask]))

    # fix parameter in fit and covariance calculation
    mask[iparam] = False
    #min_options['finite_diff_rel_step'] = steps[mask]
    err, _ = fh.calculate_covariance(fobj, p_init[mask])
    errs.append(err)
    
    # calculate impacts from up/down variations of n.p. on p.o.i.
    p_init[iparam] = pdata.val_fit + pdata.err_fit
    print(p_init[iparam], fobj(p_init[mask]))
    res = minimize(fobj, p_fit[mask],
                   jac     = fobj_jac,
                   #hess    = 'cs',
                   #method  = 'trust-constr', 
                   method  = 'BFGS', 
                   options = min_options,
                  )
    impact_up = res.x
    print((res.x[:4]-p_fit[:4])*100/p_fit[:4], res.fun)
                    
    p_init[iparam] = pdata.val_fit - pdata.err_fit
    print(p_init[iparam], fobj(p_init[mask]))
    res = minimize(fobj, p_fit[mask],
                   jac     = fobj_jac,
                   #hess    = 'cs',
                   #method  = 'trust-constr', 
                   method  = 'BFGS', 
                   options = min_options,
                  )
    impact_down = res.x
    print((res.x[:4]-p_fit[:4])*100/p_fit[:4], res.fun)
    impacts.append(np.array([impact_up, impact_down]))
    
    p_init[iparam] = pdata.val_fit
    mask[iparam] = True
    
    iparam += 1
    
errs = np.array(errs)
impacts = np.array(impacts)

4 br_tau_e 0.1774 0.0004 0.1774 0.00039995186158182927 8.464340339742221e-12


/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:182: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  f_del = np.vstack(list(np.ravel(r)) for r in sequence)
/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:184: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for step in steps)


0.17779995186158182 0.5082910003703465
         Current function value: 0.500001
         Iterations: 44
         Function evaluations: 87
         Gradient evaluations: 77
[-0.013  0.002  0.002  0.001] 0.5000007263488931
0.1770000481384182 0.5082920912217235
Optimization terminated successfully.
         Current function value: 0.500001
         Iterations: 43
         Function evaluations: 62
         Gradient evaluations: 62
[ 0.013 -0.002 -0.002 -0.001] 0.5000010626149465
5 br_tau_mu 0.173 0.0004 0.173 0.00039937223234409177 8.464340339742221e-12


/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:182: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  f_del = np.vstack(list(np.ravel(r)) for r in sequence)
/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:184: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for step in steps)


0.17339937223234408 0.5179855282360913
         Current function value: 0.500000
         Iterations: 43
         Function evaluations: 144
         Gradient evaluations: 133
[-0.001 -0.01  -0.039  0.008] 0.49999972172346646
0.1726006277676559 0.5179905746634015
         Current function value: 0.500002
         Iterations: 43
         Function evaluations: 172
         Gradient evaluations: 155
[ 0.001  0.01   0.039 -0.008] 0.5000019199893838
6 br_tau_h 0.6496 0.001 0.6496 0.000999276807314767 -4.974332057372519e-11


...

0.9844037087677752 17.483460898859427
         Current function value: 0.518359
         Iterations: 53
         Function evaluations: 168
         Gradient evaluations: 158
[-0.009 -0.015 -0.042  0.01 ] 0.5183592723096637
-0.9844037087677752 17.283477289602885
Optimization terminated successfully.
         Current function value: 0.507894
         Iterations: 57
         Function evaluations: 82
         Gradient evaluations: 82
[ 0.006  0.009  0.031 -0.007] 0.5078942454247141
115 jes_time_pt_eta 0.0 1.0 0.0 0.9941233706379774 -4.974332057372519e-11


/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:182: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  f_del = np.vstack(list(np.ravel(r)) for r in sequence)
/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:184: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for step in steps)


0.9941233706379774 1.0819803139411428
         Current function value: 0.511992
         Iterations: 46
         Function evaluations: 119
         Gradient evaluations: 107
[ 0.027  0.026  0.019 -0.012] 0.5119915192698515
-0.9941233706379774 1.054599594933651
         Current function value: 0.502235
         Iterations: 47
         Function evaluations: 122
         Gradient evaluations: 111
[-0.005 -0.007  0.02  -0.001] 0.502235084195781
116 jes_flavor_qcd 0.0 1.0 0.0 0.7323472544969002 -4.974332057372519e-11


/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:182: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  f_del = np.vstack(list(np.ravel(r)) for r in sequence)
/home/naodell/opt/anaconda3/lib/python3.7/site-packages/numdifftools/limits.py:184: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  for step in steps)


0.7323472544969002 264.86910676995353
         Current function value: 0.518660
         Iterations: 69
         Function evaluations: 151
         Gradient evaluations: 139
[ 0.177  0.172  0.243 -0.095] 0.5186597924489188
-0.7323472544969002 273.17067931894763
         Current function value: 0.531129
         Iterations: 68
         Function evaluations: 177
         Gradient evaluations: 166
[-0.195 -0.172 -0.301  0.107] 0.5311291889358123



In [7]:
# convert impacts to dataframes 
impacts_np = np.hstack([100*(impacts[:,0,:4] - p_fit[:4])/p_fit[:4], 100*(impacts[:,1,:4] - p_fit[:4])/p_fit[:4]])

#impacts_up = pd.DataFrame(impacts_np, columns=['beta_e_up', 'beta_mu_up', 'beta_tau_up', 'beta_h_up'], index=list(p_labels_fancy[4:]))
p_labels = parameters.query('active == 1').index.values[4:]
impacts_np = pd.DataFrame(impacts_np, columns=['beta_e_up', 'beta_mu_up', 'beta_tau_up', 'beta_h_up', 'beta_e_down', 'beta_mu_down', 'beta_tau_down', 'beta_h_down'], index=list(p_labels))
impacts_np.to_csv('local_data/impacts_asimov.csv')

pd.set_option('display.float_format', lambda x: '%.4f' % x)
pd.set_option('display.max_columns', parameters.shape[0])
pd.set_option('display.max_rows', parameters.shape[0])

var = 'beta_tau'
sorted_ind = impacts_np.abs().sort_values(by=f'{var}_down', ascending=False).index
impacts_np.reindex(sorted_ind)[['beta_tau_up', 'beta_tau_down']]

,beta_tau_up,beta_tau_down
eff_tau_5,-0.7982,0.7831
eff_tau_4,-0.6541,0.6337
eff_tau_2,-0.5545,0.5776
eff_tau_3,-0.5347,0.5512
xs_zjets_alt_qcd_scale_0,0.4791,-0.5398
isr,0.4014,-0.4859
top_pt,0.5104,-0.4374
xs_ttbar_qcd_scale,-0.4385,0.4279
misid_tau_h,-0.3634,0.3664
xs_t,0.3315,-0.3562


In [8]:
# convert to errors
err_no_bb = parameters['err_no_bb'].values[mask]
errs_np = np.array([np.concatenate([e[:i+4], [0], e[i+4:]]) for i, e in enumerate(errs)])
errs_np = err_no_bb**2 - errs_np**2
#errs_np[errs_np < 0] = 0
#errs_np = np.sqrt(errs_np)
##errs_np = np.vstack([errs_np, err_stat, err_mc_stat, err_syst, stderr])
#
errs_np = pd.DataFrame(errs_np[:,:4], columns=['beta_e', 'beta_mu', 'beta_tau', 'beta_h'], index=list(p_labels))
##errs_np = pd.DataFrame(errs_np[:,:4], columns=['beta_e', 'beta_mu', 'beta_tau', 'beta_h'], index=list(p_labels_fancy[4:]) + ['stat.', 'MC stat.', 'syst. total', 'total'])
##beta_stderr = stderr.iloc[:,:4].multiply(100)/params_init[:4]
100*errs_np

,beta_e,beta_mu,beta_tau,beta_h
br_tau_e,-0.0000,-0.0000,-0.0001,-0.0002
br_tau_mu,-0.0000,-0.0000,-0.0001,-0.0002
br_tau_h,-0.0000,-0.0000,-0.0001,-0.0002
lumi,-0.0000,-0.0000,-0.0001,-0.0002
xs_diboson,-0.0000,-0.0000,-0.0001,-0.0002
xs_ww,-0.0000,-0.0000,-0.0001,-0.0002
xs_t,-0.0000,-0.0000,-0.0001,-0.0001
xs_wjets,-0.0000,-0.0000,-0.0001,-0.0002
e_fakes,-0.0000,-0.0000,-0.0001,-0.0002
mu_fakes,-0.0000,-0.0000,-0.0001,-0.0002


In [9]:
# print table

#beta_errs.divide(params_init[:4]/100, axis=1).to_latex('local_data/errors.tex')
#beta_errs.divide(params_init[:4]/100, axis=1).to_csv('local_data/errors.csv')
#beta_errs.divide(params_init[:4]/100, axis=1)
beta_errs.to_latex('local_data/summary_errors.tex', escape=False)
beta_errs

NameError: name 'beta_errs' is not defined

In [ ]:
jes_mask = np.array([True if ('jes' in pname and 'btag' not in pname) else False for pname in beta_errs.index])
btag_mask = np.array([True if 'btag' in pname else False for pname in beta_errs.index])
tau_misid_mask = np.array([True if ('misid_tau' in pname and pname not in ['misid_tau_e', 'misid_tau_h']) else False for pname in beta_errs.index])

btag_errs = beta_errs[btag_mask]
jes_errs = beta_errs[jes_mask]
tau_misid_errs = beta_errs[tau_misid_mask]

summary_errs = beta_errs[~btag_mask&~jes_mask&~tau_misid_mask].copy()
summary_errs.index = [fit_data._parameters.loc[p].label if p in fit_data._parameters.index else p for p in summary_errs.index]
summary_errs.loc['b-tag',:] = np.sqrt(np.sum(btag_errs**2))
summary_errs.loc['JES',:]  = np.sqrt(np.sum(jes_errs**2))
summary_errs.loc[r'$\sf jet\rightarrow\tau$',:]  = np.sqrt(np.sum(tau_misid_errs**2))

summary_errs = summary_errs.divide(params_init[:4]/100, axis=1)
summary_errs.to_latex('local_data/summary_errors.tex', escape=False)
summary_errs.to_csv('local_data/summary_errors.csv')
summary_errs